## Lab 1. Parsing code

#### add necessary librari

In [1]:
pip install psycopg2-binary

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 8.4 MB/s eta 0:00:00:00:010:01
Note: you may need to restart the kernel to use updated packages.


In [2]:
!pip install pandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 14.2 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 9.7 MB/s eta 0:00:00:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 341.8/341.8 kB 38.2 MB/s eta 0:00:00


#### Parser part

In [3]:
import pandas as pd
import json
import warnings
from typing import List
import os

import requests
from bs4 import BeautifulSoup
import pandas as pd
from tqdm import tqdm

import numpy as np

import time
import psycopg2

In [11]:
def kp_get_recommends(query: str) -> bytes:
    url = "https://www.kinopoisk.ru/s/type/film/list/1/find/"
    response = requests.get(url + query)
    if response.status_code == 200:
        return response.content
    else:
        raise Exception(
            f"Bad status received!\n"
            f"Code: {response.status_code}\n"
            f"Content: {response.content}"
        )

def get_film_details(content):
    film_name = content.find("p", {"class": "name"})
    
    film_a_entity = film_name.find("a", {"data-type": ["film", "series"]})
    film_span_entity = film_name.find("span", {"class": "year"})
    
    film_info = content.find("span", {"class": "gray"})
    
    gray_info = None if film_info is None else film_info.getText()
    
    minutes = None
    if gray_info is not None:
        gray_info = gray_info.strip()
        if gray_info[-3:] == 'мин':
            
            if len(gray_info.split(', ')) == 1:  # only diration, no original name
                original_name = ''                
                minutes = gray_info.split(' ')[0]
            else:  # case with original name and diration
                original_name = ', '.join(gray_info.split(', ')[:-1])
                minutes = gray_info.split(', ')[-1:][0].split(' ')[0]
            
        else:
            original_name = gray_info # case with no duration and with or without original name
            minutes = ''
                
    return {
            'film_name': film_a_entity.getText().replace('\xa0',' '),
            'kinopoisk_id': film_a_entity["data-id"],
            'film_year': None if film_span_entity is None else film_span_entity.getText(),
            'original_name': original_name.replace('\xa0',' '),
            'duration': minutes
    }
    
    
        
def kp_parse_recommends(byte_response: bytes, cutoff) -> List[str]:
    
    result = []
    soup = BeautifulSoup(byte_response.decode("UTF-8"), 'html.parser')
    
    search_results_first = soup.find("div", {"class": "most_wanted"})    
    search_results_rest = soup.find("div", {"class": "search_results search_results_last"})

    if search_results_first is not None:
        for f_ind, film in enumerate(search_results_first.find_all("div", {"class": "info"})):
            film_details = get_film_details(film)
            film_details['position'] = f_ind + 1
            result.append(film_details) 

    first_block = len(result)
    
    if search_results_rest is not None:
        for r_ind, film in enumerate(search_results_rest.find_all("div", {"class": "info"})):
            film_details = get_film_details(film)
            film_details['position'] = first_block + r_ind + 1
            result.append(film_details) 

    if len(result) == 0:
        # warnings.warn("kp recommendations is empty!")
        return None

    return result[:cutoff]


def kp_querying_pipeline(query: str, cutoff=10) -> List[str]:
    raw_response = kp_get_recommends(query)
    #print(raw_response)
    return kp_parse_recommends(raw_response, cutoff=cutoff)

In [12]:
#kp_querying_pipeline('карты деньги')

In [13]:
class GrabKinopoisk:
    def __init__(
        self,  
        grab_func, 
        output_df_name, 
        chunk_size=500,        
        items_per_query=10, 
        random_delay_delta = 0.5,
        verbose=True
    ):
        self.chunk_size = chunk_size
        self.grab_func = grab_func
        self.output_df = output_df_name + '_{}' + '.csv'
        self.items_per_query = items_per_query
        self.random_delay_delta = random_delay_delta
        self.verbose=True

        
    def __grab_query(self, query):
        #res = self.grab_func(query)
        delay = 0.5 + np.random.uniform(0, self.random_delay_delta)
        time.sleep(delay)
        # res = temp_res
        res = self.grab_func(query)
        if res is not None:
            res_for_df = {
                'query': [query] * len(res),
                'title': [r['film_name'] for r in res],
                'year' : [r['film_year'] for r in res],
                'original_name': [r['original_name'] for r in res],
                'kinopoisk_id': [r['kinopoisk_id'] for r in res],
                'duration': [r['duration'] for r in res],
                'position': [r['position'] for r in res],
            }
        else:
            res_for_df = {
                'query': [query],
                'title': [''],
                'year' : [''],
                'original_name': [''],
                'kinopoisk_id': [''],
                'duration': [''],
                'position': [''],
            }

            
        return res_for_df

    
    def __grab_chunk(self, query_portion):
        self.res_dict = {'query': [], 
                    'title': [], 
                    'year': [], 
                    'original_name': [], 
                    'kinopoisk_id': [], 
                    'duration': [], 
                    'position': [] 
                   }
        for i, query in enumerate(query_portion):
            res = self.__grab_query(query)
            for k,v in res.items():
                self.res_dict[k] += v

            if (i % 10 == 0) and (self.verbose):
                print(f'\t{i}-th query: {query}')
        
        return pd.DataFrame(self.res_dict).sort_values(['query','position'])

    
    def grab_all(self, queries_series, stat_chunk_no=0, n_chunks_grab=None):
        queries_n = len(queries_series)
        n_chunks = queries_n // self.chunk_size + (1 if (queries_n % self.chunk_size > 0) else 0)
        print(f'{queries_n=}')
        print(f'{n_chunks=}')
        
        if n_chunks_grab:
            end_chunks_no = min(stat_chunk_no + n_chunks_grab, n_chunks)
        else:
            end_chunks_no = n_chunks
            
        print(f'\nfirst chunk: {stat_chunk_no}')
        print(f' last chunk: {end_chunks_no}\n')
 
            
        for i in range(stat_chunk_no, end_chunks_no):
            if self.verbose:
                print(f'{i}-th chunk:')
            start_time = time.time()    
            query_portion = queries_series[i*self.chunk_size: i*self.chunk_size + self.chunk_size]
            self.chunk_df = self.__grab_chunk(query_portion)
            chunk_df_name = self.output_df.format(i)
            self.chunk_df.to_csv(chunk_df_name)
            end_time = time.time() 
            if self.verbose:
                print(f'\t{chunk_df_name} saved')
                print(f'\ttime consumed: {int(end_time - start_time) // 60}:{int(end_time - start_time) % 60}')



In [14]:
queries_df = pd.read_csv('query_examples.csv', index_col=0)
queries_df.head()

,query
0,13 воин
1,13 грехов
2,13 ж
3,13 жи
4,13 жиз


In [15]:
queries_df.shape

(300, 1)

In [ ]:
gk = GrabKinopoisk(
    kp_querying_pipeline,
    './data/2023_04_07_kinopoisk_respond',
    chunk_size=50
)

gk.grab_all(
    queries_df['query'], 
    stat_chunk_no=0, 
    n_chunks_grab=6
)

queries_n=300
n_chunks=6

first chunk: 0
 last chunk: 6

0-th chunk:
	0-th query: 13 воин
	10-th query: 13 карт
	20-th query: 13 п
	30-th query: 13 часов
	40-th query: 1612 хро
	./data/2023_04_07_kinopoisk_respond_0.csv saved
	time consumed: 1:10
1-th chunk:
	0-th query: 18 14
	10-th query: 191
	20-th query: 1995
	30-th query: 2 плюс 3 в пользу
	40-th query: 2001 a space travesty
	./data/2023_04_07_kinopoisk_respond_1.csv saved
	time consumed: 1:20
2-th chunk:
	0-th query: 2021 уб
	10-th query: 21 мост
	20-th query: 25 7
	30-th query: 3 000 желаниййй
	40-th query: 3 богатыря и ход у
	./data/2023_04_07_kinopoisk_respond_2.csv saved
	time consumed: 1:10
3-th chunk:
	0-th query: 3 м
	10-th query: 3 тысячи
	20-th query: 300 спар
	30-th query: 3000 ет
	40-th query: 309
	./data/2023_04_07_kinopoisk_respond_3.csv saved
	time consumed: 1:9
4-th chunk:
	0-th query: 365 д
	10-th query: 4 k ultra
	20-th query: 4 эшелон
	30-th query: 5 к
	40-th query: 5 минут тишины симбирские морозы 9 серия
	./da

## Lab 2. Store data to postgres db

In [106]:
conn = psycopg2.connect(
    host="db",
    database="db",
    user="postgres",
    port="5432",
    password="az_lab_db")

#### Create table of not exist

In [107]:
table_name = 'kinopoisk'
create_table_sql = [
    f'DROP TABLE IF EXISTS "{table_name}"',
    f'DROP SEQUENCE IF EXISTS kinopoisk_id_seq',
    f'CREATE SEQUENCE kinopoisk_id_seq INCREMENT 1 MINVALUE 1 MAXVALUE 2147483647 CACHE 1',
    f"""CREATE TABLE public.{table_name} (
        "id" integer DEFAULT nextval('kinopoisk_id_seq') NOT NULL,
        "query" character varying(80) NOT NULL,
        "title" character varying(160) NOT NULL,
        "year" character varying(36) NULL,
        "original_name" character varying(160) NULL,
        "kinopoisk_id" integer NULL,
        "duration" character varying NULL,
        "position" integer NULL,
        CONSTRAINT "kinopoisk_pkey" PRIMARY KEY ("id")
    ) WITH (oids = false);
    """
]

cur = conn.cursor()
for cmd in create_table_sql:
    print('Executed:\n\t',cmd)
    cur.execute(cmd)
conn.commit()
cur.close()

Executed:
	 DROP TABLE IF EXISTS "kinopoisk"
Executed:
	 DROP SEQUENCE IF EXISTS kinopoisk_id_seq
Executed:
	 CREATE SEQUENCE kinopoisk_id_seq INCREMENT 1 MINVALUE 1 MAXVALUE 2147483647 CACHE 1
Executed:
	 CREATE TABLE public.kinopoisk (
        "id" integer DEFAULT nextval('kinopoisk_id_seq') NOT NULL,
        "query" character varying(80) NOT NULL,
        "title" character varying(160) NOT NULL,
        "year" character varying(36) NULL,
        "original_name" character varying(160) NULL,
        "kinopoisk_id" integer NULL,
        "duration" character varying NULL,
        "position" integer NULL,
        CONSTRAINT "kinopoisk_pkey" PRIMARY KEY ("id")
    ) WITH (oids = false);
    


#### Inserting data to DB

In [108]:
sql_pattern = """
INSERT into kinopoisk (query, title, year, original_name, kinopoisk_id, duration, position)
VALUES ('{}', '{}','{}','{}',{},'{}',{})
"""

In [109]:
def save_df_to_db(df):
    cur = conn.cursor()
    for d in df.itertuples(index=False):

        sql = sql_pattern.format(
                d.query,
                d.title,
                d.year,
                d.original_name,
                int(d.kinopoisk_id),
                d.duration,
                int(d.position)
        )
        cur.execute(sql)
        conn.commit()

    cur.close()
    
def clear_df(df):
    df['title'] = df['title'].str.replace("'", '`')
    df['original_name'] = df['original_name'].str.replace("'", '`')
    df['kinopoisk_id'] = df['kinopoisk_id'].fillna(0)
    df['position'] = df['position'].fillna(0)    
    return df


In [110]:
fname_pattern = '2023_04_07_kinopoisk_respond'
data_path = './data'
dir_all = os.listdir(data_path)
matched_files = [(f, int(f.split('.csv')[0].split('_')[-1])) for f in dir_all if f.startswith(fname_pattern)]
matched_files = sorted(matched_files, key=lambda x: x[1])
matched_files

[('2023_04_07_kinopoisk_respond_0.csv', 0),
 ('2023_04_07_kinopoisk_respond_1.csv', 1),
 ('2023_04_07_kinopoisk_respond_2.csv', 2),
 ('2023_04_07_kinopoisk_respond_3.csv', 3),
 ('2023_04_07_kinopoisk_respond_4.csv', 4),
 ('2023_04_07_kinopoisk_respond_5.csv', 5)]

In [111]:
for csv_name, n in matched_files:
    
    df = pd.read_csv(data_path + '/' + csv_name, index_col=0)  
    df = clear_df(df)
    print(f'processing `{csv_name}` with shape = {df.shape}')
    save_df_to_db(df)

processing `2023_04_07_kinopoisk_respond_0.csv` with shape = (385, 7)
processing `2023_04_07_kinopoisk_respond_1.csv` with shape = (403, 7)
processing `2023_04_07_kinopoisk_respond_2.csv` with shape = (322, 7)
processing `2023_04_07_kinopoisk_respond_3.csv` with shape = (350, 7)
processing `2023_04_07_kinopoisk_respond_4.csv` with shape = (357, 7)
processing `2023_04_07_kinopoisk_respond_5.csv` with shape = (351, 7)


In [ ]:
cur = conn.cursor()
cur.execute('select * from public.kinopoisk')
record = cur.fetchall()
print("Data from Database:- ", record)

#### Checking results

In [126]:
cur = conn.cursor()
cur.execute('select * from public.kinopoisk')
records = cur.fetchall()
#print("Data from Database:- ", record)
data_ind = {'id': [row[0] for row in records],}
data = {
    
    'query': [row[1] for row in records],
    'title': [row[2] for row in records],
    'year': [row[3] for row in records],
    'original_name': [row[4] for row in records],
    'kinopoisk_id': [row[5] for row in records],
    'duration': [row[6] for row in records],
    'position': [row[7] for row in records],
}
df = pd.DataFrame(data, index=data_ind['id'])
print(f'{df.shape = }')
df.head(10)

df.shape = (2168, 7)


,query,title,year,original_name,kinopoisk_id,duration,position
1,13 воин,13-й воин,1999,The 13th Warrior,10273,102.0,1
2,13 воин,13 убийц,2010,Jusan-nin no shikaku,466582,141.0,2
3,13 воин,Хоббит: Битва пяти воинств,2014,The Hobbit: The Battle of the Five Armies,694633,144.0,3
4,13 воин,Легенда о Коловрате,2017,nan,844118,117.0,4
5,13 воин,Солдаты 13 (сериал),2007,nan,542481,44.0,5
6,13 воин,Дружина (сериал),2015,nan,840738,90.0,6
7,13 воин,Хоббит: Нежданное путешествие,2012,The Hobbit: An Unexpected Journey,278522,169.0,7
8,13 воин,Храброе сердце,1995,Braveheart,399,178.0,8
9,13 воин,Брат воина,2002,Le frère du guerrier,51490,115.0,9
10,13 воин,Золотая орда (сериал),2018 – ...,nan,780307,nan,10
